In [27]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from IPython.display import Markdown

In [28]:
load_dotenv()
os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="llama-3.3-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000259A7C0F100>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000259A80743E0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [29]:
hf_api = os.getenv("HUGGINGFACE_API_KEY")
embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [30]:
loader = WebBaseLoader(web_path="https://en.wikipedia.org/wiki/World_War_I")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=385)
splits = text_splitter.split_documents(docs)


In [31]:
vector_db = Chroma.from_documents(splits,embeddings)
vector_db

In [32]:
retriever = vector_db.as_retriever()

In [ ]:
system_prompt = (
    "You are an assistant for question answering tasks. "
    "Use the following piece of retrieved context to answer the question. "
    "If you do not know the answer, say that you do not know. "
    "Use three sentences maximum. Keep the answer concise.\n{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

response = rag_chain.invoke({
    "input":"what caused the world war 1"
})
print(response['answer'])

The causes of World War I included the rise of the German Empire and decline of the Ottoman Empire, which disturbed the balance of power in Europe, the exacerbation of imperial rivalries, and an arms race between the great powers. The immediate trigger was the assassination of Franz Ferdinand, the heir to the Austro-Hungarian throne, by Gavrilo Princip, a Bosnian Serb, on 28 June 1914. This event led to a chain reaction of diplomatic crises and military mobilizations that eventually resulted in the outbreak of war.


The text does not provide a specific number of soldiers that Russia had. However, it mentions that Allied personnel, which includes Russia, was around 42,928,000, and that Russia lost 2.5-3.5 million soldiers as prisoners. Additionally, Russian casualties totalled nearly five million killed, wounded or captured by the end of 1916.
